<h1 align="center">Setting Up</h1>

In [1]:
#set up; import necessary packages/libraries
import os
#!pip install sodapy
from sodapy import Socrata
import pandas as pd
from datetime import datetime
import calendar
import warnings
warnings.filterwarnings("ignore")

#!pip install --upgrade google-cloud-storage
#!pip install --upgrade google-cloud-bigquery
#!pip install pandas-gbq
from google.cloud import bigquery
from pandas.io import gbq


#make sure to set environment var for GOOGLE_APPLICATION_CREDENTIALS
#https://cloud.google.com/bigquery/docs/quickstarts/quickstart-client-libraries
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = CREDENTIALS

<h1 align="center">Extract & Transform for Collisions Dataset</h1>

## Extracting - Collisions Data

In [2]:
#credentials needed to pull data from the API
domain = "data.cityofnewyork.us"
collisions_data_id = "h9gi-nx95"
token = "TOKEN"

#set up a connection using the credentials
client_collisions = Socrata(domain,token)

In [3]:
#check metadata; make sure we're pulling the right dataset
#also helps with familiarizing ourselves with the dataset
metadata_collisions = client_collisions.get_metadata(collisions_data_id)

#metadata #<- view metadata

In [4]:
#query applied when pulling the data from the api
#filter to only pull records older than Dec 31, 2018; start from 2019
collisions_query = """
    select collision_id,
        crash_date, 
        crash_time, 
        borough, 
        zip_code,
        number_of_persons_injured, number_of_persons_killed,
        number_of_pedestrians_injured, number_of_pedestrians_killed,
        number_of_cyclist_injured, number_of_cyclist_killed,
        number_of_motorist_injured, number_of_motorist_killed,
        contributing_factor_vehicle_1, contributing_factor_vehicle_2, contributing_factor_vehicle_3,
        vehicle_type_code1, vehicle_type_code2, vehicle_type_code_3
    where crash_date > '2018-12-31'
    limit 100000000
"""

In [5]:
#pull data from the API
#pass in query towards the request
pull_collisions_data = client_collisions.get(collisions_data_id, query = collisions_query)

In [6]:
#create a dataframe for the collisions data pulled from the API
collisions_df = pd.DataFrame(pull_collisions_data)

In [7]:
#replace all NaN values with Unspecified
collisions_df.fillna("Unspecified", inplace = True)

In [8]:
#preview data; look at head, info, and shape of dataset
#collisions_df.head()
collisions_df.info()
collisions_df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425829 entries, 0 to 425828
Data columns (total 19 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   collision_id                   425829 non-null  object
 1   crash_date                     425829 non-null  object
 2   crash_time                     425829 non-null  object
 3   number_of_persons_injured      425829 non-null  object
 4   number_of_persons_killed       425829 non-null  object
 5   number_of_pedestrians_injured  425829 non-null  object
 6   number_of_pedestrians_killed   425829 non-null  object
 7   number_of_cyclist_injured      425829 non-null  object
 8   number_of_cyclist_killed       425829 non-null  object
 9   number_of_motorist_injured     425829 non-null  object
 10  number_of_motorist_killed      425829 non-null  object
 11  contributing_factor_vehicle_1  425829 non-null  object
 12  contributing_factor_vehicle_2  425829 non-nu

(425829, 19)

## Transforming - Collisions Data

In [9]:
#convert crash_date column to datetime 
collisions_df["crash_date"] = pd.to_datetime(collisions_df["crash_date"])
collisions_df["crash_time"] = pd.to_datetime(collisions_df["crash_time"])

In [10]:
#create Year, Month, and Week columns; extract from crash_date col
collisions_df["year"] = collisions_df["crash_date"].dt.year

collisions_df["month"] = collisions_df["crash_date"].dt.month
collisions_df["month"] = collisions_df["month"].apply(lambda x: calendar.month_abbr[x])

collisions_df["week"] = collisions_df["crash_date"].dt.isocalendar().week
collisions_df["week"] = collisions_df["week"].astype(int) #<---- convert dtype from UINT32 to int64; will cause issues with pyarrow if we don't

In [11]:
collisions_df.head()

,collision_id,crash_date,crash_time,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,...,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2,borough,zip_code,contributing_factor_vehicle_3,vehicle_type_code_3,year,month,week
0,4060927,2019-01-01,2021-12-03 17:00:00,0,0,0,0,0,0,0,...,Unspecified,Sedan,Sedan,Unspecified,Unspecified,Unspecified,Unspecified,2019,Jan,1
1,4129434,2019-01-01,2021-12-03 01:30:00,1,0,0,0,0,0,1,...,Unspecified,Station Wagon/Sport Utility Vehicle,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,2019,Jan,1
2,4060684,2019-01-01,2021-12-03 01:55:00,0,0,0,0,0,0,0,...,Unspecified,Taxi,Unspecified,Unspecified,Unspecified,Unspecified,Unspecified,2019,Jan,1
3,4060563,2019-01-01,2021-12-03 00:20:00,0,0,0,0,0,0,0,...,Unspecified,Station Wagon/Sport Utility Vehicle,Sedan,QUEENS,11377,Unspecified,Unspecified,2019,Jan,1
4,4060689,2019-01-01,2021-12-03 03:30:00,1,0,0,0,0,0,1,...,Unspecified,Sedan,Station Wagon/Sport Utility Vehicle,QUEENS,11103,Unspecified,Station Wagon/Sport Utility Vehicle,2019,Jan,1


In [12]:
#create function that will denote if timestamp is AM or PM
def period_converter(x):
    hour = x.hour
    if hour < 12:
        period ="AM"
    else:
        period ="PM"
        
    return period

#create Period column; apply period_converter to all rows in crash_time column
collisions_df["period"] = collisions_df["crash_time"].apply(lambda x: period_converter(x))

In [13]:
#dictionary containing current col names and what they will be changed to
col_rename = {"number_of_persons_injured": "persons_injured",
             "number_of_persons_killed": "persons_killed",
             "number_of_pedestrians_injured": "pedestrians_injured",
             "number_of_pedestrians_killed": "pedestrians_killed",
             "number_of_cyclist_injured": "cyclists_injured",
             "number_of_cyclist_killed": "cyclists_killed",
             "number_of_motorist_injured": "motorists_injured",
             "number_of_motorist_killed": "motorists_killed",
             "vehicle_type_code_3": "vehicle_type_code3",
             "contributing_factor_vehicle_1":"contributing_factor_vehicle1",
             "contributing_factor_vehicle_2":"contributing_factor_vehicle2",
             "contributing_factor_vehicle_3":"contributing_factor_vehicle3"}

#rename columns in the df
collisions_df.rename(columns = col_rename, inplace = True)

In [14]:
#rearrange columns
collisions_df = collisions_df.reindex(columns=["collision_id",
                              "crash_date", "crash_time",
                              "year","month","week", "period",
                              "borough", "zip_code",
                              "contributing_factor_vehicle1", "contributing_factor_vehicle2", "contributing_factor_vehicle3",
                              "vehicle_type_code1", "vehicle_type_code2","vehicle_type_code3",
                               "persons_injured", "persons_killed",
                               "pedestrians_injured","pedestrians_killed",
                               "cyclists_injured","cyclists_killed",
                               "motorists_injured", "motorists_killed"
                              ])

In [15]:
collisions_df.head()

,collision_id,crash_date,crash_time,year,month,week,period,borough,zip_code,contributing_factor_vehicle1,...,vehicle_type_code2,vehicle_type_code3,persons_injured,persons_killed,pedestrians_injured,pedestrians_killed,cyclists_injured,cyclists_killed,motorists_injured,motorists_killed
0,4060927,2019-01-01,2021-12-03 17:00:00,2019,Jan,1,PM,Unspecified,Unspecified,Unspecified,...,Sedan,Unspecified,0,0,0,0,0,0,0,0
1,4129434,2019-01-01,2021-12-03 01:30:00,2019,Jan,1,AM,Unspecified,Unspecified,Alcohol Involvement,...,Unspecified,Unspecified,1,0,0,0,0,0,1,0
2,4060684,2019-01-01,2021-12-03 01:55:00,2019,Jan,1,AM,Unspecified,Unspecified,Unspecified,...,Unspecified,Unspecified,0,0,0,0,0,0,0,0
3,4060563,2019-01-01,2021-12-03 00:20:00,2019,Jan,1,AM,QUEENS,11377,Following Too Closely,...,Sedan,Unspecified,0,0,0,0,0,0,0,0
4,4060689,2019-01-01,2021-12-03 03:30:00,2019,Jan,1,AM,QUEENS,11103,Fatigued/Drowsy,...,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,1,0,0,0,0,0,1,0


### Creating Dimension Tables (Collisions/Crashes)

In [16]:
"""
    Creating dimension table ids
"""

#get index and use the numbers for numerical portion of the dimension table ids
index_nums_collisions =collisions_df.index.tolist()

#create function for generating id col for dim tables
#df = dim table dataframe, denoter = letter portion of id, id_col_name = name of id col, index_list = list with index #s from df
def dim_id_generator(df,denoter,id_col_name,index_list): #id_col_name is a str
    id_col_name_str = str(f'{id_col_name}')
    table_id_list = [f"{denoter}{x+1}" for x in index_list]
    df[id_col_name_str] = table_id_list
    df = df
    
    return df

In [17]:
#create date dim table
dim_crashdate = collisions_df[["year", "month", "week", "period"]]

#create location dimtable
dim_crashlocation = collisions_df[["borough", "zip_code"]]

#create contributing factor dim table
dim_contributingfactor = collisions_df[["contributing_factor_vehicle1", "contributing_factor_vehicle2", 
                                       "contributing_factor_vehicle3",
                                       "vehicle_type_code1", "vehicle_type_code2","vehicle_type_code3"]]
#create casualties dim table
dim_casualties = collisions_df[["persons_killed","persons_injured",
                                "pedestrians_killed","pedestrians_injured",
                                "cyclists_killed", "cyclists_injured",
                                "motorists_killed", "motorists_injured"]]

In [18]:
#generate id col for the dimension tables

dim_id_generator(dim_casualties,"CA","dim_casualties_id", index_nums_collisions)
dim_id_generator(dim_contributingfactor,"CF","dim_contributingfactor_id", index_nums_collisions)
dim_id_generator(dim_crashdate, "CD","dim_crashdate_id", index_nums_collisions)
dim_id_generator(dim_crashlocation,"CL", "dim_crashlocation_id",index_nums_collisions);

In [19]:
#rearrange columns in the dim table dataframes
dim_tables_list = [dim_crashdate,dim_crashlocation,dim_contributingfactor, dim_casualties]

#function for rearranging columns in the dim table dataframes
#dim_table = dim table df, id_col_name = col header name for id col
def column_arranger(dim_table, id_col_name):
    first_col = dim_table.pop(id_col_name)
    dim_table.insert(0,id_col_name,first_col)
    return dim_table

In [20]:
#rearrange cols so that id col is first col
column_arranger(dim_crashdate, "dim_crashdate_id")
column_arranger(dim_crashlocation, "dim_crashlocation_id")
column_arranger(dim_contributingfactor, "dim_contributingfactor_id")
column_arranger(dim_casualties, "dim_casualties_id");


<h1 align="center">Extract & Transform for 311 Traffic Lights Complaints</h1>

## Extracting - 311 Traffic Lights Complains Data

In [21]:
#credentials needed to pull data from the API
domain = "data.cityofnewyork.us"
complaints311_id = "erm2-nwe9"
token = "TOKEN"

#set up a connection using the credentials
client_complaints311 = Socrata(domain,token)

In [22]:
trafficlights311_query = """
    select unique_key, complaint_type, descriptor, address_type,
        borough, incident_zip,
        created_date
    where created_date > '2018-12-31'
    and complaint_type = "Street Light Condition"
    or complaint_type = "Traffic Light Condition"
    limit 100000000
"""

In [23]:
#pulling traffic lights 311 dataset
pull_trafficlights311_data = client_complaints311.get(complaints311_id, query = trafficlights311_query)

In [24]:
#create dataframe of the trafficlights 311 data pulled from API
#replace NaN Values with "Unspecified"
trlights311_df = pd.DataFrame(pull_trafficlights311_data)

In [25]:
#preview data; look at head, info, and shape of dataset
#trlights311_df.head()
trlights311_df.info()
trlights311_df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185438 entries, 0 to 185437
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   unique_key      185438 non-null  object
 1   complaint_type  185438 non-null  object
 2   descriptor      185438 non-null  object
 3   address_type    185438 non-null  object
 4   borough         168370 non-null  object
 5   incident_zip    108169 non-null  object
 6   created_date    185438 non-null  object
dtypes: object(7)
memory usage: 9.9+ MB


(185438, 7)

## Transforming - 311 Traffic Lights Complaints Data

In [26]:
#replace all NaN values with "Unspecified"
trlights311_df.fillna("Unspecified", inplace = True)

In [27]:
trlights311_df.head()

,unique_key,complaint_type,descriptor,address_type,borough,incident_zip,created_date
0,46145538,Street Light Condition,Lamppost Knocked Down,INTERSECTION,BROOKLYN,11207,2020-05-06T13:31:00.000
1,41458201,Street Light Condition,Street Light Out,INTERSECTION,BROOKLYN,11231,2019-01-18T17:38:00.000
2,50149364,Street Light Condition,Street Light Out,INTERSECTION,QUEENS,11365,2021-03-29T12:51:00.000
3,50152400,Street Light Condition,Multiple Street Lights Out,INTERSECTION,STATEN ISLAND,10305,2021-03-29T12:01:00.000
4,50215478,Street Light Condition,Street Light Out,BLOCKFACE,Unspecified,Unspecified,2021-04-07T21:54:00.000


In [28]:
#convert created_date col to datetime
trlights311_df["created_date"] = pd.to_datetime(trlights311_df["created_date"])

In [29]:
trlights311_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185438 entries, 0 to 185437
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   unique_key      185438 non-null  object        
 1   complaint_type  185438 non-null  object        
 2   descriptor      185438 non-null  object        
 3   address_type    185438 non-null  object        
 4   borough         185438 non-null  object        
 5   incident_zip    185438 non-null  object        
 6   created_date    185438 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(6)
memory usage: 9.9+ MB


In [30]:
#create Year, Month, and Week columns; extract from created_date col
trlights311_df["year"] = trlights311_df["created_date"].dt.year

trlights311_df["month"] = trlights311_df["created_date"].dt.month
trlights311_df["month"] = trlights311_df["month"].apply(lambda x: calendar.month_abbr[x])

trlights311_df["week"] = trlights311_df["created_date"].dt.isocalendar().week
trlights311_df["week"] = trlights311_df["week"].astype(int) #<---- convert dtype from UINT32 to int64; will cause issues with pyarrow if we don't

#create Period column; use period_converter function - apply to all rows in created_time column
trlights311_df["period"] = trlights311_df["created_date"].apply(lambda x: period_converter(x))

In [31]:
trlights311_df.head()

,unique_key,complaint_type,descriptor,address_type,borough,incident_zip,created_date,year,month,week,period
0,46145538,Street Light Condition,Lamppost Knocked Down,INTERSECTION,BROOKLYN,11207,2020-05-06 13:31:00,2020,May,19,PM
1,41458201,Street Light Condition,Street Light Out,INTERSECTION,BROOKLYN,11231,2019-01-18 17:38:00,2019,Jan,3,PM
2,50149364,Street Light Condition,Street Light Out,INTERSECTION,QUEENS,11365,2021-03-29 12:51:00,2021,Mar,13,PM
3,50152400,Street Light Condition,Multiple Street Lights Out,INTERSECTION,STATEN ISLAND,10305,2021-03-29 12:01:00,2021,Mar,13,PM
4,50215478,Street Light Condition,Street Light Out,BLOCKFACE,Unspecified,Unspecified,2021-04-07 21:54:00,2021,Apr,14,PM


In [32]:
#rename incident_zip to zip_code
trlights311_df.rename(columns = {"incident_zip":"zip_code"}, inplace = True)

### Creating Dimension Tables (311 Traffic Light Complaints)

In [33]:
#create complaint dim table
dim_311complaint = trlights311_df[["complaint_type","descriptor","address_type"]]

#create complaint location dim table
dim_311location = trlights311_df[["borough","zip_code"]]

#create complaint date dim table
dim_311date = trlights311_df[["year","month","week","period"]]

In [34]:
dim_311date.head()

,year,month,week,period
0,2020,May,19,PM
1,2019,Jan,3,PM
2,2021,Mar,13,PM
3,2021,Mar,13,PM
4,2021,Apr,14,PM


In [35]:
#index of trlights_df
index_nums_trlights311 = trlights311_df.index.tolist()

#create id col for the dim tables for 311 traffic light complaints using dim_id_generator function
#df = dim table dataframe, denoter = letter portion of id, id_col_name = name of id col, index_list = list with index #s from df
#def dim_id_generator(df,denoter,id_col_name,index_list): # denoter &id_col_name are strings

dim_id_generator(dim_311complaint,"TC","dim_311complaint_id",index_nums_trlights311)
dim_id_generator(dim_311location,"TL","dim_311location_id",index_nums_trlights311)
dim_id_generator(dim_311date,"TD","dim_311date_id",index_nums_trlights311);

In [36]:
#rearranging columns in the dim table dataframes so that id col is first using column_arranger
#dim_table = dim table df, id_col_name = col header name for id col
#def column_arranger(dim_table, id_col_name): #id_col_name is a str

column_arranger(dim_311complaint, "dim_311complaint_id")
column_arranger(dim_311date, "dim_311date_id")
column_arranger(dim_311location,"dim_311location_id");


<h1 align="center">Loading Collisions/Crashes and 311 Traffic Lights Complaints to Google BigQuery</h1>

In [38]:
#instantiate bigquery client
client = bigquery.Client(project='cis4400-assignments')

In [49]:
#function for porting dim_tables to GBQ
# df = dim table, table_name = table name
def port_table_to_gbq(table_name,df): #table_name is a string
    destination_table_path = f"cis4400_finalproject.{table_name}"
    job = df.to_gbq(destination_table = destination_table_path,
             project_id = "cis4400-assignments",
             if_exists= "replace")
    return job

In [50]:
#list containing the dim tables
dim_tables_container = {"dim_311complaint":dim_311complaint, 
                        "dim_311location":dim_311location, 
                        "dim_311date":dim_311date,
                        "dim_crashdate":dim_crashdate, 
                        "dim_crashlocation":dim_crashlocation, 
                        "dim_contributingfactor":dim_contributingfactor, 
                        "dim_casualties":dim_casualties}

#port the dim tables to GBQ using the port_table_to_gbq function
for tablename,dimtable in dim_tables_container.items():
    port_table_to_gbq(tablename, dimtable)

100%|██████████| 1/1 [00:00<?, ?it/s]
